In [1]:
import os
import sys
import math
import psutil
os.environ["_CCTRACE_"]="ON"
import cloudComPy as cc
from gendata import getSampleCloud, dataDir
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import laspy
import tqdm
from time import sleep
from pathlib import Path
print("Main : Libs loaded")

Main : Libs loaded


## Plans

In [ ]:
# TODO Load the data (using laspy -> cloudComPy)
# TODO optional convert into pandas dataframe (pandas limit to 100mb?)
# TODO Test functions:
# - getPointCloud
# - getSphere
# 
# TODO Slice the data
# TODO Subtract the data, need to ease the surface with gaussian fucntion, search the net for examples
# TODO Visualize the data
""" Current plan:
        - I cannot see what the functions from cloudComPy are doing(No peek definition), there is only documentation on webstite with no exapmles of functions alone.
                There are test cases, however they are not commented or documented on how they are achieving the results.
                And for now I don't see clear solution there on how to obtain the difference between two point clouds
        - I am creating my own point-clouds for the tests, I need to know how to create them (The libary accepts data in very specific way).
                I have made tests for reading the data from a file, but neither .las nor .LiData files are supported in a native way
                    Solution to that is to use additional libary called "laspy" which is a wrapper for las file format.(And then the data is in diffrent format than the normal data from cloudCompPy)
                Reason being that I need small sets of data and visualize them to see if the algorythms work.
                I want to see how it is visiualized and how it is subtracted on e.g. two point-cloud spheres. (I want to see if the difference is correct)
                If it works on small dataset it will also work on larger dataset.
                I cannot use the whole dataset with the mountain because it is too big.( No clear indication of the difference will be visible at this point in time.)
        - It requires theory/heavy algebra to understand how the algorythms work. Octree, Hausdorff distance, scalar fields etc.
                Even if these functions are present in libary the data must be prepared accordingly to use them. Even more so if I will use my own point cloud data.
        - Steps:
            1 Create a point-cloud spheres with the data
            2 Make a mesh/field form cloud-points(?) (The best way would be to calculate the the distance on the point-cloud and then use the distances to create scalar field, it would give the colours to the diffrenece)
            3 Use the Hausdorff distance algo to calculate the difference between the two spheres
            4 If it returns the point-cloud with the difference, then it is done.
            5 Apply to a big dataset
        - I might be missing something from the documentation that could do the job much faster, but I am not an expert on point-clouds.
        - Any suggestions/remarks are welcome
        
"""
""" Plan with Emanuele:
        1       Read the point cloud with Laspy
        2       Convert the point cloud to a cloudComPy format (A text file with three float numbers per line)
        3       Read the file with cloudComPy
        4       Create a mesh from the point cloud / repeat for the second point cloud
        5       Make a boolean INTERSECTION operation of the two meshes
        5.1     Calculate the Hausdorff distance between the two meshes (using cloudComPy) (Maybe use distance function form cloudComPy)
        6       Visualize the difference between the two meshes as one mesh
        7       Convert the mesh to a verticies only
        8       Convert the verticies to a point cloud and save it as a file

    Problem:
        1       The Hausdorff distance is not working properly. (it might not give the output that we desire)
        2       The information to which point-cloud does a point(Verticies) belong to is lost.
        2.1     It is possible to compare XYZ of a point from the difference Mesh of two point-clouds to points of the said clouds.
        2.2     But it about 70_000 points per cloud right now so that gives a lot of overhead.
        2.3     Need to find a solution for "height" of the difference mesh.

"""

"CloudCompare Workflow"
"""
The first thing is to align the 2 point clouds in order to measure only true changes (not the noise of the point clouds). There are 2 options to do this.
The first one Align two clouds by picking is if your clouds are not georeferenced. The second one is Finely registers etc. (ICP).
You will have to crop the both point clouds to keep the only stable part and apply this second option on the cropped clouds.
This gives you the transformation matrix you will have to apply on one of the point cloud (depending on which one you have aligned the other).
Once the two point clouds are aligned, you only have to calculate the distance between the two in order to have a point cloud of the changes (distances). 
You can use either one or the other point cloud as a reference, or you can define a new point cloud regularly spaced to store the distances.
The pluggin to do this is already included in cloudcompare.
It is called m3c2. You only have to export the distance point cloud as a raster at the end (interpolation can be also done within a GIS if you prefer but it is much much faster within cloudcompare)
        by choosing m3c2 distance as active layer to export.
"""

## Paths to files (To be rewritten for CLI)

In [2]:
path1 = r"C:\\Users\\szinp\\Desktop\\lazorScan_project\\testData\\cloud_e4d00e9_CutResult_17_7_42.LiData"
path2 = r"C:\\Users\\szinp\\Desktop\\lazorScan_project\\testData\\cloud_e4d00e9_CutResult_17_14_7.las"
path3 = r"C:\\Users\\szinp\\Desktop\\lazorScan_project\\testData\\cloud_e4d00e9_CutResult_17_14_7.LiData"
path4 = r"C:\\Users\\szinp\\Desktop\\lazorScan_project\\testData\\laser_1_2021-06-01-09-54-35_color_CutResult_17_14_7.las"
path5 = r"C:\\Users\\szinp\\Desktop\\lazorScan_project\\testData\\laser_1_2021-06-01-09-54-35_color_CutResult_17_14_7.LiData"
path6 = r"C:\Users\szinp\Desktop\lazorScan_project\testData\1.las"
path7 = r"C:\Users\szinp\Desktop\lazorScan_project\testData\2.las"
assert os.path.isfile(path1)
assert os.path.isfile(path2)
assert os.path.isfile(path3)
assert os.path.isfile(path4)
assert os.path.isfile(path5)
assert os.path.isfile(path6)
assert os.path.isfile(path7)
# with open(file=path, mode="r") as cloudFile:
#     print("Main : File opened")
print("All paths are good")

All paths are good


In [ ]:
#* os.environ["_CCTRACE_"]="ON"                                           # only if you want debug traces from C++

#* import cloudComPy as cc                                                # import the CloudComPy module

#* cloud = cc.loadPointCloud("myCloud.xyz")                               # read a point cloud from a file
#* print("cloud name: %s"%cloud.getName())

#* res=cc.computeCurvature(cc.CurvatureType.GAUSSIAN_CURV, 0.05, [cloud]) # compute curvature as a scalar field
#* nsf = cloud.getNumberOfScalarFields()
#* sfCurv=cloud.getScalarField(nsf-1)
#* cloud.setCurrentOutScalarField(nsf-1)
#* filteredCloud=cc.filterBySFValue(0.01, sfCurv.getMax(), cloud)         # keep only the points above a given curvature

#* ok = filteredCloud.exportCoordToSF(False, False, True)                 # Z coordinate as a scalar Field
#* nsf = cloud.getNumberOfScalarFields()
#* sf1=filteredCloud.getScalarField(nsf-1)
#* mean, var = sf1.computeMeanAndVariance()

#* # using Numpy...

#* coordinates = filteredCloud.toNpArrayCopy()                            # coordinates as a numpy array
#* x=coordinates[:,0]                                                     # x column
#* y=coordinates[:,1]
#* z=coordinates[:,2]

#* f=(2*x-y)*(x+3*y)                                                      # elementwise operation on arrays

#* asf1=sf1.toNpArray()                                                   # scalar field as a numpy array
#* sf1.fromNpArrayCopy(f)                                                 # replace scalar field values by a numpy array

#* res=cc.SavePointCloud(filteredCloud,"myModifiedCloud.bin")             #save the point cloud to a file

In [9]:
def convertLasTxt(pathToFile : str, nameOfFileOutput : str = "myCloud.txt") -> list:
    """convertLasTxt Converts .las file to .txt file for reading with cloudComPy.
    Note: it uses tqdm to show the progress of the conversion.

    Parameters
    ----------
    pathToFile : str
        _description_
    """
    listCoord_XYZ = np.empty([1,3])
    tmpList = []
    x:float = 0.0
    y:float = 0.0
    z:float = 0.0
    #! remark - path 2 has a lot of points, so it takes a while to read it (76_568_359 to be exact).
    with tqdm.tqdm(total=Path(pathToFile).stat().st_size) as pbar:
        with laspy.open(source = pathToFile) as cloudFileHeader:
            if type(cloudFileHeader) != None:
                cloudFile = laspy.read(source=path2)
                try:
                    print("Cloud loaded")
                    print(f"cloud name:{cloudFile.header}")
                except:
                    print(f"cloud name:<unknown>")
                    
            #? It seems that chunk_iterator helps to read data in batches. In every batch it reads the data and then it moves to the next batch.
            #? Points.array is in fact a one point array. if we set chunk_iterator to (1)
            #? What is the data inside of the point? It must be x,y,z and something else. It is possible to extract x,y,z from the point. Is it enough data?
            for points in cloudFileHeader.chunk_iterator(1):
                x,y,z = points.array[0]['X'], points.array[0]['Y'], points.array[0]['Z']
                valuesToAppend = np.array([[x,y,z]])
                tmpList.append(valuesToAppend)
                # listCoord_XYZ = np.concatenate((listCoord_XYZ, valuesToAppend), axis=0)
                # np.append(arr = listCoord_XYZ, values= valuesToAppend , axis=0)
                pbar.update(n=1)
            listCoord_XYZ = np.array(tmpList) 
            pbar.close()
            np.savetxt(nameOfFileOutput, listCoord_XYZ, delimiter=" ", newline="\n")
            print(f"convertLasTxt _> File {nameOfFileOutput} saved")
    print("convertLasTxt _> Conversion finished")
    return listCoord_XYZ

### Create a sphere pointcloud txt for testing

```python

In [52]:
# pass radius as an argument
# TODO: Make a hemisphere instead of a sphere

def create_points_around_atom(number:int, center:list, radius:float, cutTheShape : bool = True) -> list:

    # generate the random quantities
    phi         = np.random.uniform( 0, 2*np.pi, size=(number,))
    theta_cos   = np.random.uniform(-1,       1, size=(number,))
    u           = np.random.uniform( 0,       1, size=(number,))

    # calculate sin(theta) from cos(theta)
    theta_sin   = np.sqrt(1 - theta_cos**2)
    r           = radius * np.cbrt(u)

    # use list comprehension to generate the coordinate array without a loop
    # don't forget to offset by the atom's position (center)
    return np.array([
        np.array([
            center[0] + r[i] * theta_sin[i] * np.cos(phi[i]),
            center[1] + r[i] * theta_sin[i] * np.sin(phi[i]),
            center[2] + r[i] * theta_cos[i]
        ]) for i in range(number)
    ])

def createSphere(radius:float = 5, number:int=10,  center:list = [0,0,0], cutTheShape : str = "None", leftTreshold : float = 1, rightTreshold : float = 1) -> list:
    pi = math.pi
    sin = math.sin
    cos = math.cos
    lst = []
    for phi in [(pi*i)/(number-1) for i in range(number)]:
        M = int(sin(phi)*(number-1))+1
        for theta in [(2*pi*i)/M for i in range(M)]:
            x = center[0] + radius * sin(phi) * cos(theta)
            y = center[1] + radius * sin(phi) * sin(theta)
            z = center[2] + radius * cos(phi)
            if (cutTheShape == "left" and z<=leftTreshold):
                continue

            elif (cutTheShape == "right" and z>=rightTreshold):
                continue

            else:
                lst.append((x, y, z))

    return np.array(lst)
    # return np.array([
    #     np.array([
    #         center[0] + radius   * sin(phi) * cos(theta),
    #         center[1] + radius   * sin(phi) * sin(theta),
    #         center[2] + radius   * cos(phi)
    #     ]) for i in range(number)
    # ])

In [58]:
# sphereCoords1 = create_points_around_atom(number=10000, center=[0,0,0], radius=5)
sphereCoords1 = createSphere(number=100, center=[0,0,0], radius=5, cutTheShape="left", leftTreshold = 1)
sphereCoords2 = createSphere(number=100, center=[0,0,7], radius=6, cutTheShape="right", rightTreshold =5)
print(sphereCoords1.shape)
print(sphereCoords1)
print(sphereCoords2.shape)
print(sphereCoords2)
np.savetxt("testSphere_1.txt", sphereCoords1, delimiter=" ", newline="\n")
np.savetxt("testSphere_2.txt", sphereCoords2, delimiter=" ", newline="\n")

(22874, 3)
[[ 0.00000000e+00  0.00000000e+00  5.00000000e+00]
 [ 5.25340276e-02  0.00000000e+00  4.99972401e+00]
 [ 3.21678144e-18  5.25340276e-02  4.99972401e+00]
 ...
 [ 4.88529352e+00 -3.14719670e-01  1.01728007e+00]
 [ 4.89091872e+00 -2.09893548e-01  1.01728007e+00]
 [ 4.89429488e+00 -1.04970909e-01  1.01728007e+00]]
(19123, 3)
[[ 5.65356553e+00  0.00000000e+00  4.99072233e+00]
 [ 5.65216228e+00  1.25955535e-01  4.99072233e+00]
 [ 5.64795322e+00  2.51848544e-01  4.99072233e+00]
 ...
 [-6.30408331e-02  7.72027545e-18  1.00033119e+00]
 [-1.15804132e-17 -6.30408331e-02  1.00033119e+00]
 [ 7.34788079e-16  0.00000000e+00  1.00000000e+00]]


### Reading the first .LAS file
[]: # Language: python
[]: # Path: substractionPointCloudTest.ipynb

In [8]:
listOfCloudPointsFirst = convertLasTxt(pathToFile=path6
, nameOfFileOutput="myCloud_1.txt")

  0%|          | 0/1524029258 [00:00<?, ?it/s]

Cloud loaded
cloud name:<LasHeader(1.4, <PointFormat(7, 0 bytes of extra dims)>)>


  0%|          | 65226/1524029258 [02:24<938:50:39, 450.90it/s] 


KeyboardInterrupt: 

In [ ]:
# Using path2 as input (This is .las data file)
# cc.initCC()



# listCoord_XYZ = np.array([], ndmin = 2)
# listCoord_XYZ = np.empty([1,3])
# x:float = 0.0
# y:float = 0.0
# z:float = 0.0
# limit = 0
# #! remark - path 2 has a lot of points, so it takes a while to read it (76_568_359 to be exact).
# with tqdm.tqdm(total=Path(path6).stat().st_size) as pbar:
#     with laspy.open(source = path6) as cloudFileHeader:
#         if type(cloudFileHeader) != None:
#             cloudFile = laspy.read(source=path2)
#             try:
#                 print("Cloud loaded")
#                 print(f"cloud name:{cloudFile.header}")
#             except:
#                 print(f"cloud name:<unknown>")
#         # limit = 0
#         #? It seems that chunk_iterator helps to read data in batches. In every batch it reads the data and then it moves to the next batch.
#         #? Points.array is in fact a one point array. if we set chunk_iterator is set to (1)
#         #? What is the data inside of the point? It must be x,y,z and something else. It is possible to extract x,y,z from the point. Is it enough data?

#         for points in cloudFileHeader.chunk_iterator(1):
#             # print("type(points)\n", type(points))
#             # print("points _>\n", points)
#             # print("points array _>\n", points.array)
#             # print("type(points.array[0]) _>\n", type(points.array[0]))
#             # print("points.array[0] _>\n", points.array[0])
#             # print("points.array[0]['X'] _>\n", points.array[0]['X'])

#             # print("points.dtype\n", points.dtype)
#             # print("points.dtype.names\n", points.dtype.names)
#             x,y,z = points.array[0]['X'], points.array[0]['Y'], points.array[0]['Z']
#             valuesToAppend = np.array([[x,y,z]])
#             # print("(valuesToAppend).shape",(valuesToAppend).shape)
#             # print( "listCoord_XYZ.shape", listCoord_XYZ.shape)

#             listCoord_XYZ = np.append(arr = listCoord_XYZ, values= valuesToAppend , axis=0)
#             # pbar.update(cloudFileHeader.tell() - pbar.n)
#             pbar.update(n=1)

#             # if limit == 10:
#             #     break
#             # limit += 1
#         pbar.close()
#         print("listCoord_XYZ\n", listCoord_XYZ)

### Saving the first result in a .txt file for compatibility with CloudComPy
[]: # Language: python
[]: # Path: substractionPointCloudTest.ipynb

### Reading the second .LAS file 
[]: # Language: python
[]: # Path: substractionPointCloudTest.ipynb

In [ ]:
listOfCloudPointsSecond = convertLasTxt(pathToFile=path7
, nameOfFileOutput="myCloud_2.txt")

In [ ]:
# listCoord_XYZ = np.empty([1,3])
# x:float = 0.0
# y:float = 0.0
# z:float = 0.0
# with laspy.open(source = path7) as cloudFileHeader:
#     if type(cloudFileHeader) != None:
#         cloudFile = laspy.read(source=path4)
#         try:
#             print("Cloud loaded")
#             print(f"cloud name:{cloudFile.header}")
#         except:
#             print(f"cloud name:<unknown>")
#     # limit = 0
#     #? It seems that chunk_iterator helps to read data in batches. In every batch it reads the data and then it moves to the next batch.
#     #? Points.array is in fact a one point array. if we set chunk_iterator is set to (1)
#     #? What is the data inside of the point? It must be x,y,z and something else. It is possible to extract x,y,z from the point. Is it enough data?
#     for points in cloudFileHeader.chunk_iterator(1):
#         # print("type(points)\n", type(points))
#         # print("points _>\n", points)
#         # print("points array _>\n", points.array)
#         # print("type(points.array[0]) _>\n", type(points.array[0]))
#         # print("points.array[0] _>\n", points.array[0])
#         # print("points.array[0]['X'] _>\n", points.array[0]['X'])

#         # print("points.dtype\n", points.dtype)
#         # print("points.dtype.names\n", points.dtype.names)
#         x,y,z = points.array[0]['X'], points.array[0]['Y'], points.array[0]['Z']
#         valuesToAppend = np.array([[x,y,z]])
#         # print("(valuesToAppend).shape",(valuesToAppend).shape)
#         # print( "listCoord_XYZ.shape", listCoord_XYZ.shape)

#         listCoord_XYZ = np.append(arr = listCoord_XYZ, values= valuesToAppend , axis=0)
        

#         # if limit == 10:
#         #     break
#         # limit += 1
#     print("listCoord_XYZ\n", listCoord_XYZ)

### Saving the second result in a .txt file for compatibility with CloudComPy
[]: # Language: python
[]: # Path: substractionPointCloudTest.ipynb

In [ ]:
# np.savetxt("listCoord_XYZ_2.txt", listCoord_XYZ, delimiter=" ", newline="\n")

#### TODO: Make a progress bar for computing

In [ ]:
# Done